# Exportation des donnee a mongodb et sauvegard du model

In [26]:
# Import necessary libraries
import pymongo
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib
# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["temperatureDB"]
collection = db["temperatureCollection"]
# Clear the collection if it already exists
collection.delete_many({})
# Manually enter the data (Example Data)
data = {
    "Celsius": [ce for ce in range(50)],
    "Fahrenheit": [(c * 9/5) + 32 for c in [ce for ce in range(50)]],
    "Kelvin": [c + 273.15 for c in [ce for ce in range(50)]]
}
# Convert to DataFrame
df = pd.DataFrame(data)
# Insert data into MongoDB
collection.insert_many(df.to_dict('records'))
# Fetch the data from MongoDB
data_from_db = collection.find()
df_from_db = pd.DataFrame(list(data_from_db))
df_from_db.drop('_id', axis=1, inplace=True)
# Separate features and target
X = df_from_db[['Fahrenheit', 'Kelvin']]
y = df_from_db['Celsius']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)
# Save the trained model
joblib.dump(model, 'temperature_model.pkl')
print("Model trained and saved as 'temperature_model.pkl'")


Model trained and saved as 'temperature_model.pkl'


# Prediction

In [27]:
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Print model coefficients
print(f"Coefficients: {model.coef_}")
print(f"Intercept: {model.intercept_}")


Mean Squared Error: 1.7165613297609216e-28
Coefficients: [ 0.62843766 -0.13118778]
Intercept: 15.72393787234737
